In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
cd ..

C:\Projects\python\recommender


In [2]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [11]:
from datasets import SeqTopcoder
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [4]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHAG_PATH = Path("./inputs/topcoder/challenge.csv")

In [5]:
db = SeqTopcoder(regs_path=REGS_PATH, chag_path=CHAG_PATH)
db

Read dataset in inputs\topcoder\regs.csv
Original regs shape: (610025, 3)
Original registants size: 60017
Original challenges size: 39916
Filter dataframe shape: (544568, 3)
Index(['challengeId', 'period', 'date', 'prizes', 'technologies', 'platforms'], dtype='object')


In [6]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

In [7]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Create criterion

In [8]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [9]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001B4AC69C730>, 1, 1)

In [10]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001B4AC6F3950>, 1, 1, 1)

In [12]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x000001B4AC6F39D8>, 1, 1)

In [13]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x000001B4AC6F3A60>, 1, 1, 1)

### Train model

#### Hyper-parameter

In [14]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

#### Train FM Model

In [15]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [16]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [17]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [18]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [20]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [ ]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=trans_weight_loss_callback)

In [20]:
fm_learner.fit(epoch=3,
               log_dir=get_log_dir('simple_topcoder', 'fm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 94905.21758678491
Epoch 0 step 1: training accuarcy: 0.3252
Epoch 0 step 1: training loss: 86457.55434978122
Epoch 0 step 2: training accuarcy: 0.3966
Epoch 0 step 2: training loss: 78937.53425374633
Epoch 0 step 3: training accuarcy: 0.45690000000000003


KeyboardInterrupt: 

In [26]:
del fm_model
T.cuda.empty_cache()

NameError: name 'fm_model' is not defined

### Train HRM FM Model

In [13]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [14]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [15]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [16]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [17]:
hrm_learner.fit(epoch=5,
                log_dir=get_log_dir('seq_topcoder', 'hrm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 42431.971135041786
Epoch 0 step 1: training accuarcy: 0.5425
Epoch 0 step 1: training loss: 41611.87258174788
Epoch 0 step 2: training accuarcy: 0.533
Epoch 0 step 2: training loss: 40729.178744947305
Epoch 0 step 3: training accuarcy: 0.534
Epoch 0 step 3: training loss: 39077.242215819024
Epoch 0 step 4: training accuarcy: 0.545
Epoch 0 step 4: training loss: 37973.78130915589
Epoch 0 step 5: training accuarcy: 0.55
Epoch 0 step 5: training loss: 37284.86898758583
Epoch 0 step 6: training accuarcy: 0.521
Epoch 0 step 6: training loss: 36142.27887250477
Epoch 0 step 7: training accuarcy: 0.5425
Epoch 0 step 7: training loss: 35028.941934657676
Epoch 0 step 8: training accuarcy: 0.5345
Epoch 0 step 8: training loss: 33826.59320649134
Epoch 0 step 9: training accuarcy: 0.5445
Epoch 0 step 9: training loss: 33069.860875282175
Epoch 0 step 10: training accuarcy: 0.5245
Epoch 0 step 10: training loss: 31872.833027400273
Epoch 0 step 11: training accu

Epoch 0 step 173: training loss: 1413.1483539222527
Epoch 0 step 174: training accuarcy: 0.6135
Epoch 0 step 174: training loss: 1413.161282819326
Epoch 0 step 175: training accuarcy: 0.637
Epoch 0 step 175: training loss: 1419.1655759174848
Epoch 0 step 176: training accuarcy: 0.606
Epoch 0 step 176: training loss: 1423.0336349849777
Epoch 0 step 177: training accuarcy: 0.6035
Epoch 0 step 177: training loss: 1430.4855983113669
Epoch 0 step 178: training accuarcy: 0.6245
Epoch 0 step 178: training loss: 1404.5872416932928
Epoch 0 step 179: training accuarcy: 0.609
Epoch 0 step 179: training loss: 1403.2749256895318
Epoch 0 step 180: training accuarcy: 0.612
Epoch 0 step 180: training loss: 1409.388347349875
Epoch 0 step 181: training accuarcy: 0.605
Epoch 0 step 181: training loss: 1398.170818423775
Epoch 0 step 182: training accuarcy: 0.6045
Epoch 0 step 182: training loss: 1399.647368426476
Epoch 0 step 183: training accuarcy: 0.6105
Epoch 0 step 183: training loss: 1397.15300451512

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [04:49<19:19, 289.80s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1370.000425387888
Epoch 1 step 264: training accuarcy: 0.6005
Epoch 1 step 264: training loss: 1337.3001136712044
Epoch 1 step 265: training accuarcy: 0.6325000000000001
Epoch 1 step 265: training loss: 1371.8238745968526
Epoch 1 step 266: training accuarcy: 0.584
Epoch 1 step 266: training loss: 1343.0026475911757
Epoch 1 step 267: training accuarcy: 0.6145
Epoch 1 step 267: training loss: 1348.0691573820966
Epoch 1 step 268: training accuarcy: 0.615
Epoch 1 step 268: training loss: 1365.8553596597278
Epoch 1 step 269: training accuarcy: 0.591
Epoch 1 step 269: training loss: 1352.8658934356934
Epoch 1 step 270: training accuarcy: 0.6115
Epoch 1 step 270: training loss: 1348.3941088485494
Epoch 1 step 271: training accuarcy: 0.625
Epoch 1 step 271: training loss: 1361.3921383169684
Epoch 1 step 272: training accuarcy: 0.5975
Epoch 1 step 272: training loss: 1345.3344596428212
Epoch 1 step 273: training accuarcy: 0.619
Epoch 1 step 273: trainin

...

Epoch 3 step 959: training loss: 1322.6312451263373
Epoch 3 step 960: training accuarcy: 0.6365000000000001
Epoch 3 step 960: training loss: 1331.308937026565
Epoch 3 step 961: training accuarcy: 0.625
Epoch 3 step 961: training loss: 1329.6602580650076
Epoch 3 step 962: training accuarcy: 0.6075
Epoch 3 step 962: training loss: 1337.9052071951016
Epoch 3 step 963: training accuarcy: 0.609
Epoch 3 step 963: training loss: 1345.7294448084106
Epoch 3 step 964: training accuarcy: 0.603
Epoch 3 step 964: training loss: 1333.9702016634976
Epoch 3 step 965: training accuarcy: 0.6155
Epoch 3 step 965: training loss: 1334.6999319372296
Epoch 3 step 966: training accuarcy: 0.6125
Epoch 3 step 966: training loss: 1337.7582848166253
Epoch 3 step 967: training accuarcy: 0.619
Epoch 3 step 967: training loss: 1348.795168443645
Epoch 3 step 968: training accuarcy: 0.6025
Epoch 3 step 968: training loss: 1324.883135915111
Epoch 3 step 969: training accuarcy: 0.6205
Epoch 3 step 969: training loss: 13

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [19:25<04:51, 291.18s/it]

Epoch: 4
Epoch 4 step 1052: training loss: 1317.8338241354895
Epoch 4 step 1053: training accuarcy: 0.632
Epoch 4 step 1053: training loss: 1337.295673641742
Epoch 4 step 1054: training accuarcy: 0.617
Epoch 4 step 1054: training loss: 1339.4580334156717
Epoch 4 step 1055: training accuarcy: 0.6085
Epoch 4 step 1055: training loss: 1335.1523034373586
Epoch 4 step 1056: training accuarcy: 0.6165
Epoch 4 step 1056: training loss: 1330.850338818631
Epoch 4 step 1057: training accuarcy: 0.6245
Epoch 4 step 1057: training loss: 1342.3026604404238
Epoch 4 step 1058: training accuarcy: 0.6085
Epoch 4 step 1058: training loss: 1320.9174572165612
Epoch 4 step 1059: training accuarcy: 0.634
Epoch 4 step 1059: training loss: 1335.5893587405053
Epoch 4 step 1060: training accuarcy: 0.609
Epoch 4 step 1060: training loss: 1336.6762282303
Epoch 4 step 1061: training accuarcy: 0.6095
Epoch 4 step 1061: training loss: 1329.878388692607
Epoch 4 step 1062: training accuarcy: 0.6245
Epoch 4 step 1062: tr

Epoch 4 step 1219: training accuarcy: 0.602
Epoch 4 step 1219: training loss: 1336.8399403828596
Epoch 4 step 1220: training accuarcy: 0.6205
Epoch 4 step 1220: training loss: 1338.2458758308348
Epoch 4 step 1221: training accuarcy: 0.617
Epoch 4 step 1221: training loss: 1339.3198867926476
Epoch 4 step 1222: training accuarcy: 0.617
Epoch 4 step 1222: training loss: 1323.054068040676
Epoch 4 step 1223: training accuarcy: 0.623
Epoch 4 step 1223: training loss: 1313.8534209165816
Epoch 4 step 1224: training accuarcy: 0.637
Epoch 4 step 1224: training loss: 1345.8422689262527
Epoch 4 step 1225: training accuarcy: 0.6095
Epoch 4 step 1225: training loss: 1338.5957478019843
Epoch 4 step 1226: training accuarcy: 0.615
Epoch 4 step 1226: training loss: 1341.1451612400979
Epoch 4 step 1227: training accuarcy: 0.607
Epoch 4 step 1227: training loss: 1328.603303964803
Epoch 4 step 1228: training accuarcy: 0.6165
Epoch 4 step 1228: training loss: 1347.4785473648208
Epoch 4 step 1229: training a

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [24:16<00:00, 291.35s/it]


In [18]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [19]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [20]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [21]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [22]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [23]:
prme_learner.fit(epoch=5,
                 log_dir=get_log_dir('seq_topcoder', 'prme'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 46927.16156849063
Epoch 0 step 1: training accuarcy: 0.528
Epoch 0 step 1: training loss: 46267.131020927816
Epoch 0 step 2: training accuarcy: 0.522
Epoch 0 step 2: training loss: 44997.714309480885
Epoch 0 step 3: training accuarcy: 0.508
Epoch 0 step 3: training loss: 43504.5824063122
Epoch 0 step 4: training accuarcy: 0.533
Epoch 0 step 4: training loss: 41738.84964275482
Epoch 0 step 5: training accuarcy: 0.5385
Epoch 0 step 5: training loss: 41276.64514832693
Epoch 0 step 6: training accuarcy: 0.519
Epoch 0 step 6: training loss: 40171.31425944169
Epoch 0 step 7: training accuarcy: 0.5265
Epoch 0 step 7: training loss: 38565.05773775875
Epoch 0 step 8: training accuarcy: 0.5425
Epoch 0 step 8: training loss: 38105.47435526393
Epoch 0 step 9: training accuarcy: 0.5205
Epoch 0 step 9: training loss: 36857.612616393904
Epoch 0 step 10: training accuarcy: 0.544
Epoch 0 step 10: training loss: 35801.712217204855
Epoch 0 step 11: training accuarc

Epoch 0 step 171: training loss: 1678.5417919913398
Epoch 0 step 172: training accuarcy: 0.622
Epoch 0 step 172: training loss: 1668.6281048016017
Epoch 0 step 173: training accuarcy: 0.649
Epoch 0 step 173: training loss: 1674.0068939601797
Epoch 0 step 174: training accuarcy: 0.628
Epoch 0 step 174: training loss: 1647.0013929377897
Epoch 0 step 175: training accuarcy: 0.6155
Epoch 0 step 175: training loss: 1633.9893575118667
Epoch 0 step 176: training accuarcy: 0.64
Epoch 0 step 176: training loss: 1635.0645925763888
Epoch 0 step 177: training accuarcy: 0.6345000000000001
Epoch 0 step 177: training loss: 1614.1180871273991
Epoch 0 step 178: training accuarcy: 0.6525
Epoch 0 step 178: training loss: 1629.1897962537098
Epoch 0 step 179: training accuarcy: 0.6285000000000001
Epoch 0 step 179: training loss: 1592.4721621138196
Epoch 0 step 180: training accuarcy: 0.6495
Epoch 0 step 180: training loss: 1602.3045706193288
Epoch 0 step 181: training accuarcy: 0.6495
Epoch 0 step 181: tra

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [04:54<19:37, 294.48s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1391.3742324237164
Epoch 1 step 264: training accuarcy: 0.6265000000000001
Epoch 1 step 264: training loss: 1386.7022409815424
Epoch 1 step 265: training accuarcy: 0.625
Epoch 1 step 265: training loss: 1375.0216078713688
Epoch 1 step 266: training accuarcy: 0.62
Epoch 1 step 266: training loss: 1397.4907678312215
Epoch 1 step 267: training accuarcy: 0.615
Epoch 1 step 267: training loss: 1390.1119370827646
Epoch 1 step 268: training accuarcy: 0.6125
Epoch 1 step 268: training loss: 1389.4605440270732
Epoch 1 step 269: training accuarcy: 0.63
Epoch 1 step 269: training loss: 1369.4495607659544
Epoch 1 step 270: training accuarcy: 0.6325000000000001
Epoch 1 step 270: training loss: 1381.6163563773262
Epoch 1 step 271: training accuarcy: 0.621
Epoch 1 step 271: training loss: 1384.3328648470238
Epoch 1 step 272: training accuarcy: 0.606
Epoch 1 step 272: training loss: 1379.2392102268109
Epoch 1 step 273: training accuarcy: 0.628
Epoch 1 step 273

...

Epoch 3 step 957: training loss: 1352.3688497546002
Epoch 3 step 958: training accuarcy: 0.621
Epoch 3 step 958: training loss: 1327.487915427688
Epoch 3 step 959: training accuarcy: 0.6175
Epoch 3 step 959: training loss: 1338.5488080333375
Epoch 3 step 960: training accuarcy: 0.6125
Epoch 3 step 960: training loss: 1351.4304674442187
Epoch 3 step 961: training accuarcy: 0.6085
Epoch 3 step 961: training loss: 1339.3711208486607
Epoch 3 step 962: training accuarcy: 0.6275000000000001
Epoch 3 step 962: training loss: 1335.2871693526122
Epoch 3 step 963: training accuarcy: 0.6245
Epoch 3 step 963: training loss: 1356.1102947082109
Epoch 3 step 964: training accuarcy: 0.613
Epoch 3 step 964: training loss: 1355.3995785741363
Epoch 3 step 965: training accuarcy: 0.606
Epoch 3 step 965: training loss: 1337.9661806828433
Epoch 3 step 966: training accuarcy: 0.621
Epoch 3 step 966: training loss: 1346.6871976498844
Epoch 3 step 967: training accuarcy: 0.6165
Epoch 3 step 967: training loss: 

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [19:30<04:52, 292.86s/it]

Epoch: 4
Epoch 4 step 1052: training loss: 1347.1714314488063
Epoch 4 step 1053: training accuarcy: 0.6175
Epoch 4 step 1053: training loss: 1345.0089515616135
Epoch 4 step 1054: training accuarcy: 0.617
Epoch 4 step 1054: training loss: 1334.2267697433642
Epoch 4 step 1055: training accuarcy: 0.6295000000000001
Epoch 4 step 1055: training loss: 1335.8911666886925
Epoch 4 step 1056: training accuarcy: 0.632
Epoch 4 step 1056: training loss: 1343.3458264172255
Epoch 4 step 1057: training accuarcy: 0.6175
Epoch 4 step 1057: training loss: 1333.918976856889
Epoch 4 step 1058: training accuarcy: 0.628
Epoch 4 step 1058: training loss: 1339.7717908624807
Epoch 4 step 1059: training accuarcy: 0.6195
Epoch 4 step 1059: training loss: 1340.1125604707786
Epoch 4 step 1060: training accuarcy: 0.6365000000000001
Epoch 4 step 1060: training loss: 1350.3484880805643
Epoch 4 step 1061: training accuarcy: 0.604
Epoch 4 step 1061: training loss: 1341.2255989429673
Epoch 4 step 1062: training accuarcy:

Epoch 4 step 1217: training accuarcy: 0.6465
Epoch 4 step 1217: training loss: 1334.6964352926996
Epoch 4 step 1218: training accuarcy: 0.6335000000000001
Epoch 4 step 1218: training loss: 1339.2602847043083
Epoch 4 step 1219: training accuarcy: 0.616
Epoch 4 step 1219: training loss: 1347.1658443645506
Epoch 4 step 1220: training accuarcy: 0.6175
Epoch 4 step 1220: training loss: 1338.238347604399
Epoch 4 step 1221: training accuarcy: 0.632
Epoch 4 step 1221: training loss: 1336.6486489915003
Epoch 4 step 1222: training accuarcy: 0.626
Epoch 4 step 1222: training loss: 1333.2472279340363
Epoch 4 step 1223: training accuarcy: 0.642
Epoch 4 step 1223: training loss: 1361.7043932116958
Epoch 4 step 1224: training accuarcy: 0.615
Epoch 4 step 1224: training loss: 1346.388730199401
Epoch 4 step 1225: training accuarcy: 0.6245
Epoch 4 step 1225: training loss: 1353.1917458528292
Epoch 4 step 1226: training accuarcy: 0.612
Epoch 4 step 1226: training loss: 1358.4421844763228
Epoch 4 step 122

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [24:42<00:00, 298.80s/it]


In [24]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [25]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [26]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [27]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [28]:
trans_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=trans_loss_callback)

In [29]:
trans_learner.fit(epoch=5,
                  log_dir=get_log_dir('seq_topcoder', 'trans'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 48586.55751144697
Epoch 0 step 1: training accuarcy: 0.5135
Epoch 0 step 1: training loss: 46554.602668407955
Epoch 0 step 2: training accuarcy: 0.5315
Epoch 0 step 2: training loss: 46073.377841289795
Epoch 0 step 3: training accuarcy: 0.5375
Epoch 0 step 3: training loss: 44473.14710635728
Epoch 0 step 4: training accuarcy: 0.5365
Epoch 0 step 4: training loss: 43457.24741669615
Epoch 0 step 5: training accuarcy: 0.5215
Epoch 0 step 5: training loss: 42315.38672174617
Epoch 0 step 6: training accuarcy: 0.5225
Epoch 0 step 6: training loss: 41311.40883139911
Epoch 0 step 7: training accuarcy: 0.54
Epoch 0 step 7: training loss: 40070.476456635355
Epoch 0 step 8: training accuarcy: 0.5225
Epoch 0 step 8: training loss: 38971.2602908507
Epoch 0 step 9: training accuarcy: 0.552
Epoch 0 step 9: training loss: 37971.164600912336
Epoch 0 step 10: training accuarcy: 0.5395
Epoch 0 step 10: training loss: 36959.90561237274
Epoch 0 step 11: training accu

Epoch 0 step 173: training loss: 1151.1784287771404
Epoch 0 step 174: training accuarcy: 0.777
Epoch 0 step 174: training loss: 1116.9814151445676
Epoch 0 step 175: training accuarcy: 0.7945
Epoch 0 step 175: training loss: 1140.5678941107212
Epoch 0 step 176: training accuarcy: 0.7885
Epoch 0 step 176: training loss: 1075.3674273548922
Epoch 0 step 177: training accuarcy: 0.803
Epoch 0 step 177: training loss: 1090.0428897173276
Epoch 0 step 178: training accuarcy: 0.79
Epoch 0 step 178: training loss: 1096.2791603129133
Epoch 0 step 179: training accuarcy: 0.7905
Epoch 0 step 179: training loss: 1057.0687897856774
Epoch 0 step 180: training accuarcy: 0.8035
Epoch 0 step 180: training loss: 1068.9123489402648
Epoch 0 step 181: training accuarcy: 0.8055
Epoch 0 step 181: training loss: 1093.2883667399813
Epoch 0 step 182: training accuarcy: 0.7915
Epoch 0 step 182: training loss: 1080.424352815765
Epoch 0 step 183: training accuarcy: 0.7985
Epoch 0 step 183: training loss: 1127.1094925

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [05:30<22:01, 330.40s/it]

Epoch: 1
Epoch 1 step 263: training loss: 893.8440470362575
Epoch 1 step 264: training accuarcy: 0.8385
Epoch 1 step 264: training loss: 880.6706700027167
Epoch 1 step 265: training accuarcy: 0.8320000000000001
Epoch 1 step 265: training loss: 883.6945793379331
Epoch 1 step 266: training accuarcy: 0.8345
Epoch 1 step 266: training loss: 884.2245551809732
Epoch 1 step 267: training accuarcy: 0.8405
Epoch 1 step 267: training loss: 905.0637195152724
Epoch 1 step 268: training accuarcy: 0.8225
Epoch 1 step 268: training loss: 873.3723081387003
Epoch 1 step 269: training accuarcy: 0.8320000000000001
Epoch 1 step 269: training loss: 889.42313099631
Epoch 1 step 270: training accuarcy: 0.8335
Epoch 1 step 270: training loss: 872.8751072640475
Epoch 1 step 271: training accuarcy: 0.841
Epoch 1 step 271: training loss: 886.7884586266921
Epoch 1 step 272: training accuarcy: 0.838
Epoch 1 step 272: training loss: 874.2252321386115
Epoch 1 step 273: training accuarcy: 0.8415
Epoch 1 step 273: tra

...

Epoch 3 step 958: training loss: 777.6858480979205
Epoch 3 step 959: training accuarcy: 0.8605
Epoch 3 step 959: training loss: 743.5984169279818
Epoch 3 step 960: training accuarcy: 0.8735
Epoch 3 step 960: training loss: 752.2032128194076
Epoch 3 step 961: training accuarcy: 0.874
Epoch 3 step 961: training loss: 754.8350933604228
Epoch 3 step 962: training accuarcy: 0.871
Epoch 3 step 962: training loss: 742.3114829549371
Epoch 3 step 963: training accuarcy: 0.873
Epoch 3 step 963: training loss: 768.9480889066733
Epoch 3 step 964: training accuarcy: 0.8575
Epoch 3 step 964: training loss: 735.3288141176668
Epoch 3 step 965: training accuarcy: 0.874
Epoch 3 step 965: training loss: 730.739718401121
Epoch 3 step 966: training accuarcy: 0.877
Epoch 3 step 966: training loss: 764.4535670722067
Epoch 3 step 967: training accuarcy: 0.8645
Epoch 3 step 967: training loss: 760.6797826546256
Epoch 3 step 968: training accuarcy: 0.8595
Epoch 3 step 968: training loss: 726.0403933882269
Epoch

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [21:58<05:26, 326.53s/it]

Epoch: 4
Epoch 4 step 1052: training loss: 767.0175590315823
Epoch 4 step 1053: training accuarcy: 0.8585
Epoch 4 step 1053: training loss: 737.7048019569734
Epoch 4 step 1054: training accuarcy: 0.876
Epoch 4 step 1054: training loss: 716.4961974492793
Epoch 4 step 1055: training accuarcy: 0.879
Epoch 4 step 1055: training loss: 734.5857265296753
Epoch 4 step 1056: training accuarcy: 0.87
Epoch 4 step 1056: training loss: 704.9380041077133
Epoch 4 step 1057: training accuarcy: 0.884
Epoch 4 step 1057: training loss: 770.1127786024069
Epoch 4 step 1058: training accuarcy: 0.868
Epoch 4 step 1058: training loss: 749.0727160168962
Epoch 4 step 1059: training accuarcy: 0.8735
Epoch 4 step 1059: training loss: 759.2212353324198
Epoch 4 step 1060: training accuarcy: 0.8695
Epoch 4 step 1060: training loss: 740.2415140687274
Epoch 4 step 1061: training accuarcy: 0.879
Epoch 4 step 1061: training loss: 747.1435214728915
Epoch 4 step 1062: training accuarcy: 0.8735
Epoch 4 step 1062: training 

Epoch 4 step 1218: training accuarcy: 0.8795000000000001
Epoch 4 step 1218: training loss: 733.5058235722876
Epoch 4 step 1219: training accuarcy: 0.8805000000000001
Epoch 4 step 1219: training loss: 768.2482503724679
Epoch 4 step 1220: training accuarcy: 0.866
Epoch 4 step 1220: training loss: 747.8459485672889
Epoch 4 step 1221: training accuarcy: 0.8675
Epoch 4 step 1221: training loss: 732.8843970368218
Epoch 4 step 1222: training accuarcy: 0.8775000000000001
Epoch 4 step 1222: training loss: 731.347330084879
Epoch 4 step 1223: training accuarcy: 0.881
Epoch 4 step 1223: training loss: 749.8136272840882
Epoch 4 step 1224: training accuarcy: 0.864
Epoch 4 step 1224: training loss: 747.1137312084136
Epoch 4 step 1225: training accuarcy: 0.8775000000000001
Epoch 4 step 1225: training loss: 794.4833948802295
Epoch 4 step 1226: training accuarcy: 0.869
Epoch 4 step 1226: training loss: 727.6877375651801
Epoch 4 step 1227: training accuarcy: 0.8795000000000001
Epoch 4 step 1227: training

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [27:18<00:00, 324.59s/it]


In [30]:
del trans_model
T.cuda.empty_cache()